# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [171]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [172]:
df = pd.read_csv('previsao_de_renda.csv')
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [173]:
df = df.drop(['data_ref', 'Unnamed: 0'], axis=1)
object_cols = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']
df[object_cols] = df[object_cols].apply(lambda x: pd.Categorical(x).codes.astype('int64'))
bool_cols = ['posse_de_veiculo', 'posse_de_imovel']
df[bool_cols] = df[bool_cols].astype(int)
float_cols = ['tempo_emprego', 'qt_pessoas_residencia', 'renda']
df[float_cols] = df[float_cols].fillna(0).astype('int64')
df.head()

,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,15056,0,0,1,0,2,2,2,1,26,6,1,8060
1,9968,1,1,1,0,0,3,0,1,28,7,2,1852
2,4312,0,1,1,0,2,3,0,1,35,0,2,2253
3,10639,0,0,1,1,4,3,0,1,30,4,3,6600
4,7064,1,1,0,0,0,2,2,5,33,4,1,6475


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [174]:
#1
X = df.drop('renda', axis=1)
y = df['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [179]:
#2
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_scores = []
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
    print(f"Ridge com Alpha={alpha}: R2={r2}")
best_alpha = alphas[np.argmax(r2_scores)]
best_r2 = max(r2_scores)
print(f"Top Ridge: Alpha={best_alpha}, Top R2={best_r2}")

Ridge com Alpha=0: R2=0.2645277590706099
Ridge com Alpha=0.001: R2=0.2645277564358611
Ridge com Alpha=0.005: R2=0.2645277458964366
Ridge com Alpha=0.01: R2=0.26452773272118857
Ridge com Alpha=0.05: R2=0.2645276272805206
Ridge com Alpha=0.1: R2=0.26452749538298637
Top Ridge: Alpha=0, Top R2=0.2645277590706099


- Alpha=0 tem o maior valor com o valoar de 0.2645277590706099.

In [183]:
#3
alphas = [0.001, 0.005, 0.01, 0.05, 0.1]
r2_scores_lasso = []
for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    r2_scores_lasso.append(r2)
    print(f"Lasso Alpha={alpha}: R2={r2}")
best_alpha_lasso = alphas[np.argmax(r2_scores_lasso)]
best_r2_lasso = max(r2_scores_lasso)
print(f"Top Lasso: Alpha={best_alpha_lasso}, Top R2={best_r2_lasso}")

Lasso Alpha=0.001: R2=0.2645277822128944
Lasso Alpha=0.005: R2=0.2645278749731865
Lasso Alpha=0.01: R2=0.26452799083828094
Lasso Alpha=0.05: R2=0.26452891482625585
Lasso Alpha=0.1: R2=0.2645300657358387
Top Lasso: Alpha=0.1, Top R2=0.2645300657358387


- O modelo Lasso com Alpha=0.1

In [195]:
#4
def stepwise_selection(X, y, initial_list=[], threshold_in=0.01, threshold_out=0.05, verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(dtype=float)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adicionando: {best_feature} com p-value {best_pval:.4f}')
        model = sm.OLS(y, sm.add_constant(X[included])).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print(f'Remove: {worst_feature} com p-value {worst_pval:.4f}')
        if not changed:
            break
    return included
selected_features = stepwise_selection(X_train, y_train)
model_stepwise = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
y_pred_stepwise = model_stepwise.predict(sm.add_constant(X_test[selected_features]))
mse_stepwise = mean_squared_error(y_test, y_pred_stepwise)
r2_stepwise = r2_score(y_test, y_pred_stepwise)
print(f"R2 stepwise: {r2_stepwise}")
print(f"Erro quadrático médio stepwise: {mse_stepwise}")

Adicionando: tempo_emprego com p-value 0.0000
Adicionando: sexo com p-value 0.0000
Adicionando: idade com p-value 0.0000
Adicionando: tipo_renda com p-value 0.0000
R2 stepwise: 0.26311515828847865
Erro quadrático médio stepwise: 49635654.331169404


- Com base no valor de R² o modelo Lasso com Alpha=0.1 continua sendo o melhor modelo até agora.

In [188]:
#5
ridge_coef = Ridge(alpha=best_alpha).fit(X_train, y_train).coef_
lasso_coef = Lasso(alpha=best_alpha_lasso).fit(X_train, y_train).coef_
print(f"Coeficientes Ridge: {ridge_coef}")
print(f"Coeficientes Lasso: {lasso_coef}")

Coeficientes Ridge: [ 7.63884926e-03  5.75210007e+03  4.23491608e+01  3.53816953e+02
 -1.12669704e+02  3.24523836e+02  2.29626377e+02 -3.82243642e+01
  1.68379937e+02  6.97702047e+01  5.06839583e+02  2.08462015e+02]
Coeficientes Lasso: [ 7.63540423e-03  5.75170541e+03  4.21239386e+01  3.53257081e+02
 -1.10632013e+02  3.24460311e+02  2.29329490e+02 -3.85632215e+01
  1.68145316e+02  6.97709712e+01  5.06830105e+02  2.06716330e+02]


- O modelo Lasso com Alpha=0.1 continua sendo o melhor modelo até agora.

In [196]:
#6
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)
lin_reg_poly = LinearRegression()
lin_reg_poly.fit(X_train_poly, y_train)
y_pred_poly = lin_reg_poly.predict(X_test_poly)
r2_poly = r2_score(y_test, y_pred_poly)
print(f"R2 com características polinomiais: {r2_poly}")

R2 com características polinomiais: 0.3775611962437957


In [197]:
#7
params = {
    'max_depth': [None, 2, 4, 6, 8, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]}
grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), params, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)
best_tree_reg = grid_search.best_estimator_
y_pred_best_tree = best_tree_reg.predict(X_test)
r2_best_tree = r2_score(y_test, y_pred_best_tree)
print(f"R2 do melhor regressor da árvore: {r2_best_tree}")

R2 do melhor regressor da árvore: 0.3442253840852739
